In [1]:
class FuzzSet(object):
    """ Set of Strings to add as input to Fuzzer """
    _min_length = 0

    def __init__(self, length, valid_chars, case_sensitive=False):
        assert len(valid_chars) > 0
        self.length = length
        self.valid_chars = valid_chars
        self.case_sensitive = case_sensitive

        if type(self.length) == list:
            self._min_length = min(self.length)
        else:
            self._min_length = self.length

        self._init_sequence = [self.valid_chars[0] for char in range(self._min_length)]
        self._current_sequence = self._init_sequence
        self._create_generator()
        self._current_count = 0

    def _create_generator(self):
        output = self.chain()
        if type(self.length) == list:
            temp_generators = []
            for val in self.length:
                temp_generators.append(self._product(val))
            for gen in temp_generators:
                output = self.chain(output, gen)
            self._generator = output
        else:
            self._generator = self._product(self.length)
    
    def _product(self, length):
        final = [[]]
        l = length
        groups = [list(self.valid_chars)] * l
        for i in groups:
            final = [x+[y] for x in final for y in i]
        for k in final:
            yield ''.join(k)

    def generate_charset(self, num_values):
        stop_value = self._current_count + num_values
        while self._current_count < stop_value:
            self._current_count += 1
            print next(self._generator)

    def next_charset(self):
        try:
            self._current_charset = next(self._generator)
            return self._current_charset
        except StopIteration:
            return False

    def current_charset(self):
        return self._current_charset

    def reset_generator(self):
        self._create_generator()

    def __repr__(self):
        return "FUZZSET: {length: %s, chars: \"%s\"}" %(self.length, str(self.valid_chars))

    @staticmethod
    def cycle(iterable):
        # cycle('ABCD') --> A B C D A B C D A B C D ...
        saved = []
        for element in iterable:
            yield element
            saved.append(element)
        while saved:
            for element in saved:
                  yield element

    @staticmethod
    def chain(*iterables):
        for it in iterables:
            for element in it:
                yield element


In [2]:
class Fuzzer(object):
    """ This is where you collect your Fuzz Sets"""
    theAnswer = ''

    def __init__(self, url):
        self.url = url
        self._fuzzset = []
        self._fuzz_product = None
    
    def __repr__(self):
        return 'FUZZER CONTAINING: %s' %(self._fuzzset)

    def add_fuzzset(self, fuzzset):
        assert isinstance(fuzzset, FuzzSet)
        self._fuzzset.append(fuzzset)
        self._completed = [False for _ in range(len(self._fuzzset))]
        self._fuzz_product = locked_iterator(self.product(*[fuzz._generator for fuzz in self._fuzzset]))
    
    def show_fuzzset(self):
        print self._fuzzset
    
    @staticmethod
    def product(*args):
        pools = [tuple(pool) for pool in args] * 1 # repeat
        result = [[]]
        for pool in pools:
            result = [x+[y] for x in result for y in pool]
        for prod in result:
            yield tuple(prod)

    def generate_fuzzable(self):
        try:
            return ''.join(next(self._fuzz_product))
        except StopIteration:
            return False
    
    def send_request(self, threads=10):
        import Queue
        import requests
        from threading import Thread
        import time
        q = Queue.Queue()
        def build_queue():
            for _ in range(threads*10):
                q.put(self.generate_fuzzable())

        def try_pw():
            while not q.empty() and ThreadsWithAStopPattern.continueThreading:
                pw = q.get()
                if pw == False:
                    return False
                new_pw = self.generate_fuzzable()
                q.put(new_pw)
                r = requests.post(self.url, data={'username': 'ROB', 'password': pw})
                q.task_done()
                if r.status_code == 200:
                    Fuzzer.theAnswer = pw
                    ThreadsWithAStopPattern.continueThreading = False
                    raise Exception('Password is: ' + pw)
            return 'FAILED TO FIND PASSWORD'
        build_queue()
        for i in range(threads): # aka number of threadtex
            t1 = ThreadsWithAStopPattern(target = try_pw) # target is the above function
            t1.start() # start the thread
        while ThreadsWithAStopPattern.continueThreading == True:
            continue
        return self.theAnswer


In [3]:
import threading

class ThreadsWithAStopPattern(threading.Thread):
    """ This is a thread that also allows for a class variable
        to be reachable by all threads """
    
    allThreads = []
    continueThreading = True

    def __init__(self, target):
        super(ThreadsWithAStopPattern, self).__init__(target=target)
        self.allThreads.append(self)


In [4]:
class locked_iterator(object):
    """Locks an iterator/generator by serializing the call to the next
       method """

    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def next(self):
        with self.lock:
            return self.it.next()

In [5]:
a = FuzzSet(1, 'Ss')
b = FuzzSet(1, 'Ee3')
c = FuzzSet(1, 'cC')
d = FuzzSet(1, 'rR')
e = FuzzSet(1, 'eE3')
f = FuzzSet(1, 'tT')
g = FuzzSet(1, '!')

In [6]:
fuzzy = Fuzzer('http://localhost:5000/login')
fuzzy.add_fuzzset(a)
fuzzy.add_fuzzset(b)
fuzzy.add_fuzzset(c)
fuzzy.add_fuzzset(d)
fuzzy.add_fuzzset(e)
fuzzy.add_fuzzset(f)
fuzzy.add_fuzzset(g)

In [7]:
fuzzy.send_request(10)

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.12/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/2.7.12/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-2-1536e18798c7>", line 59, in try_pw
    raise Exception('Password is: ' + pw)
Exception: Password is: s3cREt!



's3cREt!'

In [ ]:
"""some example code for generators"""

In [12]:
def permutations_with_repetition(s):
    base = len(s)
    for n in range(base ** base):
        yield "".join(s[n // base**(base-d-1) % base] for d in range(base))

In [9]:
def cycle(iterable):
    # cycle('ABCD') --> A B C D A B C D A B C D ...
    saved = []
    for element in iterable:
        yield element
        saved.append(element)
    while saved:
        for element in saved:
              yield element

def chain(*iterables):
    # chain('ABC', 'DEF') --> A B C D E F
    for it in iterables:
        for element in it:
            yield element
            
def product(chars, length):
    final = [[]]
    l = length
    groups = [list(chars)] * l
    for i in groups:
        final = [x+[y] for x in final for y in i]
    for k in final:
        yield ''.join(k)